In [3]:
# We can use the same thing to convert any dataset
source_dir = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\Original CSV'
out_dir = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\refit.h5'

# Only change this to code if you want to convert everything again
from nilmtk.dataset_converters import convert_refit

convert_refit(source_dir, out_dir)

In [4]:
from nilmtk import DataSet
from nilmtk.utils import print_dict

# Seeing dataset metadata
refit = DataSet(out_dir)
print_dict(refit.metadata)

In [5]:
print_dict(refit.buildings)

In [6]:
import pandas as pd

# Dictionary to hold the date ranges for each house
date_ranges = {}

for house_id in refit.buildings:
    house = refit.buildings[house_id]
    
    # The entire house
    mains = house.elec.mains()
    
    # Get the first and last sample timestamps
    start = pd.to_datetime(mains.get_timeframe().start).tz_convert('UTC')
    end = pd.to_datetime(mains.get_timeframe().end).tz_convert('UTC')
    start = start + pd.Timedelta(hours=1) - pd.Timedelta(minutes=start.minute, seconds=start.second, microseconds=start.microsecond)
    end = end - pd.Timedelta(minutes=end.minute, seconds=end.second, microseconds=end.microsecond)
    
    # Store the date range for this building
    date_ranges[house_id] = (start, end)

min_date = min(date_ranges.values(), key=lambda x: x[0])[0]
max_date = max(date_ranges.values(), key=lambda x: x[1])[1]

# Output the results
print("Minimum start date across all buildings:", min_date)
print("Maximum end date across all buildings:", max_date)

Minimum start date across all buildings: 2013-09-17 23:00:00+00:00
Maximum end date across all buildings: 2015-07-10 11:00:00+00:00


In [14]:
for meter in refit.buildings[1].elec.submeters().meters:
    for appliance in meter.appliances:
        print(appliance.metadata.get('type'))

fridge
freezer
freezer
washer dryer
washing machine
dish washer
computer
television
electric space heater


In [15]:
# device types
device_types = {}

for house_id in refit.buildings:
    elec = refit.buildings[house_id].elec
    for num, meter in enumerate(elec.submeters().meters, start=1):
        #help(meter.load)
        for appliance in meter.appliances:
            if appliance.metadata.get('type') is not None:
                appliance_type = appliance.metadata.get('type')
                # Sorting them into types with (house, applianceNum) tuples
                if appliance_type not in device_types:
                    device_types[appliance_type] = [(meter.building(), num)]
                else:
                    device_types[appliance_type].append((meter.building(), num))
            else:
                print('Nothing there')

# Print out the device types and their associated meter instances
for appliance_type, house in device_types.items():
    print(f"{appliance_type}: {house}")

fridge: [(1, 1), (11, 1), (17, 1), (19, 1), (4, 1), (7, 1), (8, 1)]
freezer: [(1, 2), (1, 3), (10, 3), (13, 2), (16, 1), (17, 2), (19, 2), (3, 3), (4, 2), (6, 1), (7, 2), (7, 3), (8, 2)]
washer dryer: [(1, 4), (17, 4), (8, 3), (9, 2)]
washing machine: [(1, 5), (10, 5), (11, 3), (13, 3), (14, 3), (15, 5), (16, 4), (17, 5), (18, 2), (19, 4), (2, 2), (20, 3), (3, 6), (4, 5), (4, 6), (5, 3), (6, 2), (7, 5), (8, 4), (9, 3)]
dish washer: [(1, 6), (10, 6), (11, 4), (13, 4), (14, 4), (15, 6), (17, 6), (19, 5), (2, 3), (20, 4), (3, 5), (5, 4), (6, 3), (7, 6), (9, 4)]
computer: [(1, 7), (11, 5), (12, 4), (14, 5), (15, 7), (16, 5), (17, 7), (19, 6), (5, 5), (6, 4), (6, 9), (8, 6)]
television: [(1, 8), (10, 7), (12, 8), (13, 1), (14, 6), (15, 8), (16, 6), (16, 9), (17, 8), (18, 3), (19, 7), (2, 4), (20, 6), (3, 7), (4, 7), (5, 6), (6, 5), (7, 7), (8, 7), (9, 5)]
electric space heater: [(1, 9), (15, 3), (15, 4), (9, 9)]
food processor: [(10, 1), (10, 9), (20, 5)]
toaster: [(10, 2), (12, 7), (14, 9)

In [16]:
# Putting the devices into groups. Possibly separate by water consumption after.
# Not in this set then it is intermittent
# Add more if you notice more above in different datasets
always_on_types = set()
always_on_types.update(['fridge', 'freezer', 'fridge freezer', 'broadband router', 'pond pump'])
condensed_types = {'AlwaysOn': {}, 'Intermit': {}}

# Sorting all of the appliances into distinct categories
for appliance_type in device_types.keys():
    if appliance_type in always_on_types:
        for (house, num) in device_types[appliance_type]:
            if house not in condensed_types['AlwaysOn']:
                condensed_types['AlwaysOn'][house] = set()
                
            condensed_types['AlwaysOn'][house].add(num)
    else:
        for (house, num) in device_types[appliance_type]:
            if house not in condensed_types['Intermit']:
                condensed_types['Intermit'][house] = set()
                
            condensed_types['Intermit'][house].add(num)

# Print out the separated categories
for cat, info in condensed_types.items():
    print(f"{cat}: {info}")

AlwaysOn: {1: {1, 2, 3}, 11: {8, 1, 2}, 17: {1, 2, 3}, 19: {1, 2}, 4: {1, 2, 3}, 7: {1, 2, 3}, 8: {1, 2}, 10: {3, 4}, 13: {2, 6}, 16: {1, 2}, 3: {2, 3}, 6: {1}, 12: {1}, 14: {1}, 15: {1, 2}, 18: {1}, 2: {1}, 20: {1, 9}, 5: {1}, 9: {1}}
Intermit: {1: {4, 5, 6, 7, 8, 9}, 17: {4, 5, 6, 7, 8, 9}, 8: {3, 4, 5, 6, 7, 8, 9}, 9: {2, 3, 4, 5, 6, 7, 8, 9}, 10: {1, 2, 5, 6, 7, 8, 9}, 11: {3, 4, 5, 6, 7, 9}, 13: {1, 3, 4, 5, 7, 8, 9}, 14: {2, 3, 4, 5, 6, 7, 8, 9}, 15: {3, 4, 5, 6, 7, 8, 9}, 16: {3, 4, 5, 6, 7, 8, 9}, 18: {2, 3, 4, 5, 6, 7, 8, 9}, 19: {3, 4, 5, 6, 7, 8, 9}, 2: {2, 3, 4, 5, 6, 7, 8, 9}, 20: {2, 3, 4, 5, 6, 7, 8}, 3: {1, 4, 5, 6, 7, 8, 9}, 4: {4, 5, 6, 7, 8, 9}, 5: {2, 3, 4, 5, 6, 7, 8, 9}, 6: {2, 3, 4, 5, 6, 7, 8, 9}, 7: {4, 5, 6, 7, 8, 9}, 12: {2, 3, 4, 5, 6, 7, 8, 9}}


In [17]:
# Keep track of row processing
def process_row(row):
    return row.nlargest(k).iloc[-1]

In [18]:
corrected_mains = {}

# I want to prevent mismeasurements by adding the W of any column
for house_id in sorted(refit.buildings):
    elec = refit.buildings[house_id].elec
    mains = elec.mains()
    mains_df = pd.DataFrame(next(mains.load()))
    all_meters_df = pd.DataFrame()

    # Going through appliances
    for num, meter in enumerate(elec.submeters().meters, start=1):
        # Checking if the appliance has any larger values
        meter_df = pd.DataFrame(next(meter.load()))
        all_meters_df = pd.concat([all_meters_df, meter_df], axis=1)

    # Calculate the sum of all meters for each row
    sum_of_meters = all_meters_df.sum(axis=1)

    # Rare case where sum is still less so get the highest val iteratively
    for k in range(1, len(elec.submeters().meters) + 1):
        condition = sum_of_meters > mains_df.squeeze()  # Assuming mains_df is a single column DataFrame
        print("Total invalid rows: " + str(condition.sum()))

        # If the condition is empty we don't have to check the next highest meter
        if not condition.any():
            break

        # Add the highest val to mains
        print("Found incorrect mains sum for house " + str(house_id) + " (iter " + str(k) + ")")
        
        # This is to take advantage of the added speed of max
        if k == 1:
            mains_df[condition] += all_meters_df[condition].max(axis=1).values.reshape(-1, 1)
        else:
            mains_df[condition] += all_meters_df[condition].apply(process_row, axis=1).values.reshape(-1, 1)

    # Add to corrected
    corrected_mains[house_id] = mains_df
    print('Done checking all of house ' + str(house_id))

print('Finished checking all houses')
        

Total invalid rows: 23850
Found incorrect mains sum for house 1 (iter 1)
Total invalid rows: 224
Found incorrect mains sum for house 1 (iter 2)
Total invalid rows: 14
Found incorrect mains sum for house 1 (iter 3)
Total invalid rows: 3
Found incorrect mains sum for house 1 (iter 4)
Total invalid rows: 0
Done checking all of house 1
Total invalid rows: 28444
Found incorrect mains sum for house 2 (iter 1)
Total invalid rows: 124
Found incorrect mains sum for house 2 (iter 2)
Total invalid rows: 9
Found incorrect mains sum for house 2 (iter 3)
Total invalid rows: 0
Done checking all of house 2


KeyboardInterrupt: 

In [ ]:
# Converting the time to int with error handling. For time values
def convert_time_to_seconds(time_obj):
    return time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second

In [ ]:
import os
import numpy as np

# Directory where you want to save the CSV files
csv_dir = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\Hourly CSV\\'
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

for house_id, (start, end) in sorted_date_ranges:
    # Initialize an empty DataFrame for the building
    house_df = pd.DataFrame()

    # Access electricity data for the house
    elec = refit.buildings[house_id].elec
    mains_df = pd.DataFrame()

    # Loading the mains data and resampling
    thirty_sec_mains = corrected_mains[house_id].resample('30S').mean()
    thirty_sec_energy = thirty_sec_mains * (1/(120)) # Converting to Wh
    mains_df = thirty_sec_energy.resample('H').sum().round(3)
    
    # Two data frames for each category
    cat_df = {'AlwaysOn': pd.DataFrame(), 'Intermit': pd.DataFrame()}

    # Go through each meter
    for num, meter in enumerate(elec.submeters().meters, start=1):
        print(f"Processing appiance {num} for house {house_id}")

        # Finding the category
        for cat, house in condensed_types.items():
            if house_id in house and num in house[house_id]:
                meter_cat = cat
        try:
            # Load the meter data with hour sampling
            # TODO CHANGE TO HANDLE MULTIPLE CHUNKS
            meter_data = next(meter.load()) 
            
            if not meter_data.empty:
                # Resample to hourly data and sum the readings
                thirty_sec_app_power = meter_data.resample('30S').mean()
                thirty_sec_app_energy = thirty_sec_app_power * (1/(120)) # Converting to Wh
                hourly_data = thirty_sec_app_energy.resample('H').sum().round(3)

            # Putting the meter in a category
            if cat_df[meter_cat].empty:
                cat_df[meter_cat] = hourly_data
            else:
                cat_df[meter_cat] += hourly_data

        except Exception as e:
            print(f"Error processing Meter{num} for House {house_id}. Error: {e}")

    # Combine the category DataFrames and handle missing data
    for cat, df in cat_df.items():
        # GOTTA HANDLE THE ZERO VALUES CORRECTLY
        df.fillna(method='ffill', inplace=True)  # Replace NaNs with a forward fill
        house_df[cat] = df.sum(axis=1)  # Sum across meters within the category

    # Adding the mains data
    house_df['Total'] = mains_df
    house_df['HVAC'] = house_df['Total'] - house_df['AlwaysOn'] - house_df['Intermit']
    
    # Converting time zones only if needed
    #house_df.index = house_df.index.tz_convert('Europe/London')
    house_df['DayNum'] = house_df.index.dayofweek
    house_df['Time'] = house_df.index.time
    house_df['Time'] = house_df['Time'].apply(convert_time_to_seconds)
    house_df['Month'] = house_df.index.month

    # Constants for cycles
    seconds_in_a_day = 24 * 60 * 60
    days_in_a_week = 7
    months_in_a_year = 12
    
    # Adding sine and cos for all time varying values
    house_df['TimeSin'] = np.sin(2 * np.pi * house_df['Time'] / seconds_in_a_day)
    house_df['TimeCos'] = np.cos(2 * np.pi * house_df['Time'] / seconds_in_a_day)
    house_df['DayNumSin'] = np.sin(2 * np.pi * house_df['DayNum'] / days_in_a_week)
    house_df['DayNumCos'] = np.cos(2 * np.pi * house_df['DayNum'] / days_in_a_week)
    house_df['MonthSin'] = np.sin(2 * np.pi * house_df['Month'] / months_in_a_year)
    house_df['MonthCos'] = np.cos(2 * np.pi * house_df['Month'] / months_in_a_year)

    # Only taking the columns I want
    cols = ['DayNumSin', 'DayNumCos', 'TimeSin', 'TimeCos', 'MonthSin', 'MonthCos', 'Total', 'AlwaysOn', 'Intermit', 'HVAC']
    house_df = house_df[cols]

    # Write the combined building data to CSV
    csv_file_path = os.path.join(csv_dir, f"house{house_id}_clean.csv")
    house_df.to_csv(csv_file_path, index=True)
    print(f"Combined hourly data for house {house_id} written to CSV.")

print("Finished processing all buildings.")

In [ ]:
houses_path = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\Hourly CSV\\house'
weather_path = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\Weather CSV\\REFITWeather.csv'
output_path = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\Completed CSV\\house'

for house_id in sorted(refit.buildings):
    cur_house = houses_path + str(house_id) + '_clean.csv'
    cur_output = output_path + str(house_id) + '.csv'

    # Reading the csvs
    house_df = pd.read_csv(cur_house, index_col='Unix', parse_dates=['Unix'])
    weather_df = pd.read_csv(weather_path, skiprows=3)
    
    # Convert to datetime
    weather_df['time'] = pd.to_datetime(weather_df['time'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Europe/London')
    
    # Set datetime as the index
    weather_df.set_index('time', inplace=True)
    
    # Merge the dataframes based on the index
    merged_df = house_df.join(weather_df, how='inner')

    # Label the index
    merged_df.reset_index(inplace=True)
    merged_df.rename(columns={'index': 'Timestamp'}, inplace=True)

    # Changing the weather names
    merged_df['RealTemp'] = merged_df['temperature_2m (°C)']
    merged_df['ApparTemp'] = merged_df['apparent_temperature (°C)']
    merged_df['Humid'] = merged_df['relative_humidity_2m (%)']
    merged_df['WmoCode'] = merged_df['weather_code (wmo code)']

    # Organizing the columns
    cols = ['TimeSin', 'TimeCos', 'DayNumSin', 'DayNumCos', 'MonthSin', 'MonthCos', 'RealTemp', 'ApparTemp', 'Humid', 'WmoCode', 'Total', 'AlwaysOn', 'Intermit', 'HVAC']
    merged_df = merged_df[cols]

    # Send to csv
    merged_df.to_csv(cur_output, index=False)
    print('Done formatting house ' + str(house_id))

print('Done formatting all houses')

In [7]:
# Get metadata
meta_input_path = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\MetaData\\HouseMetaData.csv'
meta_df = pd.read_csv(meta_input_path)

# For the houses
all_houses_path = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\Completed CSV\\house'
output_path = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\Completed With Meta\\house'

# Loop through all the houses
for house_id in range(1, len(refit.buildings) + 1):
    # Meta data. Insuring it only takes one row
    house_md = meta_df[meta_df['House'] == house_id]
    house_md = house_md.iloc[0:1]
    house_md = house_md.drop(['House'], axis=1)

    # House data
    house_path = all_houses_path + str(house_id) + '.csv'
    house_data = pd.read_csv(house_path)

    # Split in two leaving the usage values
    split_position = len(house_data.columns) - 4
    first_part = house_data.iloc[:, :split_position]
    second_part = house_data.iloc[:, split_position:]

    # Copying this row accross the entire data length and concat
    repeated_md = pd.concat([house_md]*len(house_data), ignore_index=True)
    result_df = pd.concat([first_part, repeated_md, second_part], axis=1)

    house_output_path = output_path + str(house_id) + '.csv'
    result_df.to_csv(house_output_path, index=False)

    print('Added meta data for house ' + str(house_id))

Added meta data for house 1
Added meta data for house 2
Added meta data for house 3
Added meta data for house 4
Added meta data for house 5
Added meta data for house 6
Added meta data for house 7
Added meta data for house 8
Added meta data for house 9
Added meta data for house 10
Added meta data for house 11
Added meta data for house 12
Added meta data for house 13
Added meta data for house 14
Added meta data for house 15
Added meta data for house 16
Added meta data for house 17
Added meta data for house 18
Added meta data for house 19
Added meta data for house 20


In [8]:
# Rem
input_path = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\Completed With Meta\\house'
output_path = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\Regression Training Set\\house'

# Go through all houses
for house_id in sorted(refit.buildings):
    cur_input = input_path + str(house_id) + '.csv'
    cur_output = output_path + str(house_id) + '.csv'

    # Read csv
    house_df = pd.read_csv(cur_input)

    # Make conditional series
    condition = house_df['Total'].squeeze() != 0

    # Getting rid of extra rows
    house_df = house_df[condition]
    house_df.to_csv(cur_output, index=False)
    print("Formatted house " + str(house_id))

Formatted house 1
Formatted house 2
Formatted house 3
Formatted house 4
Formatted house 5
Formatted house 6
Formatted house 7
Formatted house 8
Formatted house 9
Formatted house 10
Formatted house 11
Formatted house 12
Formatted house 13
Formatted house 14
Formatted house 15
Formatted house 16
Formatted house 17
Formatted house 18
Formatted house 19
Formatted house 20
